Парсер docdoc.ru.

Часть 1, собирает ссылки и часть информации — адрес, телефоны, часы работы и т.п.

Классы на doc_doc "кривые", поэтому в основном использую теги.

Иногда филиалы клиники находятся в самой карточке, из-за этого код повторяю. Код можно упростить, но работает и так. 

Последние правки скрипта: 2022-01-06.

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service=s, options=opt)

#driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки ##

In [ ]:
# нужный город латиницей, транслитом как на сайте

need_city = input()

In [ ]:
# ссылка на начальную страницу поиска клиник


if need_city == 'msk':
    url_main = 'https://docdoc.ru/clinic/spec/stomatologiya'
    
else:
    url_main = f'https://{need_city}.docdoc.ru/clinic/spec/stomatologiya'

In [ ]:
# открываю, чтобы посчитать количество клиник и страниц

driver.get(url_main)

In [ ]:
clinic_number = int(driver
                    .find_element_by_tag_name('span[data-test-id="clinic-list-page-header__count"]')
                    .text)
page_number = math.ceil(clinic_number / 10) # округление в большую сторону

print(f'Количество клиник: {clinic_number}')
print(f'Количество страниц: {page_number}')

In [ ]:
# ссылка на страницу каталога, пагинация в конце

url_part_1 = url_main + '/page/'

### Код парсера ###

In [ ]:
data = [] # список для данных

for page in tqdm(range(1, page_number + 1)):
    
    driver.get(url_part_1 + str(page))
    sleep(3)

    # блок с информацией о клинике
    ClinicCard_blocs = driver.find_elements_by_tag_name('article[data-test-id="clinic-card"]')
    
    for item in ClinicCard_blocs:
        
        try:
            actual = (item
                      .find_element_by_class_name('ClinicCardNoContract__root_2Zs-.ClinicCardInfo__no-contract_I20q')
                      .text)

        except:
            actual = 1


        # специализация
        try:
            speciality = item.find_element_by_tag_name('span[data-test-id="clinic-card-main-top__speciality"]').text
            
        except:
            speciality = 'none'

        # название клиники
        main_clinic = item.find_element_by_tag_name('a[data-test-id="clinic-card-main-top__clinic-name-link"]').text
        branch_clinic = main_clinic

        # ссылка на стницу клиники
        clinic_page = (item
                       .find_element_by_tag_name('a[data-test-id="clinic-card-main-top__clinic-name-link"]')
                       .get_attribute('href'))

        # тип клиники
        try:
            clinic_type = item.find_element_by_tag_name('span[data-test-id="clinic-card-main-top__institution"]').text
            
        except:
            clinic_type = 'none'
        

        # адрес
        try:
            address = item.find_element_by_tag_name('span[data-test-id="clinic-card-info__address"]').text
            
        except:
            address = 'none'

        # ближайшие станции метро
        metro_stations = {} # словарь для станций
        
        try:
            metro_station_block = item.find_elements_by_tag_name('a[data-test-id="clinic-card-info__station"]') # блок
            
            for i in metro_station_block: # цикл для разбора
                metro_stations[i.find_element_by_class_name('SubwayStation__label_1Q4O').text] = (
                    i.find_element_by_class_name('SubwayStation__dist_dDCM').text)
        except:
            pass
        

        # часы работы
        try:
            work_hours = item.find_element_by_class_name('ClinicCardInfo__schedule_2xfN').text.replace('\n', ' ')

        except:
            work_hours = 'none'

        # телефон
        
        try:
            item.find_element_by_class_name('ClinicCardInfo__phone-button_uoPy').click()
            
        except:
            pass
        
        try:
            phone = item.find_element_by_tag_name('span[data-test-id="clinic-card-info__phone"]').text
            
        except:
            phone = 'none'

        # рейтинг
        try:
            rating = (item
                      .find_element_by_class_name('rating-stars__pink')
                      .get_attribute('style')
                      .replace('width: ', '').replace(';', ''))
            
        except:
            rating = 'none'

        # рейтинг филиала
        branch_rating = rating

        
        # количество отзывов
        try:
            rewiev_number = (item
                             .find_element_by_tag_name('a[data-test-id="clinic-card-logo__reviews-amount"]')
                             .get_attribute('textContent').strip())
            
        except:
            rewiev_number = 'none'

        # количесво отзывов о филиале
        branch_rewiev_number = rewiev_number    

        
        

        # данные филиалов, если они есть
        clinic_branch_block = item.find_elements_by_tag_name('div[data-test-id="clinic-card-branch"]')

        if len(clinic_branch_block) > 0:

            # нажимаю кнопку "Показать все филиалы"
            try:
                WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CLASS_NAME, 'ClinicCardBranches__button_2XcN')))

                item.find_element_by_class_name('ClinicCardBranches__button_2XcN').click()

            except:
                pass

            clinic_branch_block = item.find_elements_by_tag_name('div[data-test-id="clinic-card-branch"]')

            # количество филиалов
            branch_number = len(clinic_branch_block)

            # разбираю

            for clinic_branch in clinic_branch_block:
                
                # название филиала
                branch_clinic = clinic_branch.find_element_by_tag_name('h5').text
                
                
                # ссылка на страницу клиники
                clinic_page = (clinic_branch
                       .find_element_by_tag_name('a[data-test-id="clinic-card-branch__clinic-name-link"]')
                       .get_attribute('href'))

                # адрес
                try:
                    address = clinic_branch.find_element_by_tag_name('span[data-test-id="clinic-card-info__address"]').text

                except:
                    address = 'none'


                # ближайшие станции метро
                metro_stations = {} # словарь для станций
                try:

                    metro_station_block = (clinic_branch
                                           .find_elements_by_tag_name('a[data-test-id="clinic-card-info__station"]')) # блок

                    for i in metro_station_block: # цикл для разбора
                        metro_stations[i.find_element_by_class_name('SubwayStation__label_1Q4O').text] = (
                            i.find_element_by_class_name('SubwayStation__dist_dDCM').text)

                except:
                    pass


                # часы работы
                try:
                    work_hours = clinic_branch.find_element_by_class_name('ClinicCardInfo__schedule_2xfN').text.replace('\n', ' ')

                except:
                    work_hours = 'none'

                # рейтинг
                try:
                    branch_rating = (clinic_branch
                                     .find_element_by_class_name('rating-stars__pink')
                                     .get_attribute('style')
                                     .replace('width: ', '').replace(';', ''))
                    
                except:
                    branch_rating = 'none'

                # количество отзывов
                try:
                    branch_rewiev_number = (clinic_branch
                                            .find_element_by_tag_name('span[data-test-id="clinic-card-branch__reviews-amount"]')
                                            .get_attribute('textContent').strip())
                    
                except:
                    branch_rewiev_number = 'none'



                data.append([main_clinic, actual,
                             speciality, clinic_type,
                             branch_clinic, branch_number,
                             address, metro_stations,
                             work_hours, phone,
                             rating, rewiev_number,
                             branch_rating, branch_rewiev_number,
                             clinic_page])


        else:
            # количество филиалов
            branch_number = 1

            data.append([main_clinic, actual,
                         speciality, clinic_type,
                         branch_clinic, branch_number,
                         address, metro_stations,
                         work_hours, phone,
                         rating, rewiev_number,
                         branch_rating, branch_rewiev_number,
                         clinic_page])

In [ ]:
# Записываю в датафрейм

head_list = ['main_clinic', 'actual',
             'speciality', 'clinic_type',
             'branch_clinic', 'branch_number',
             'address', 'metro_stations',
             'work_hours', 'phone',
             'rating', 'rewiev_number',
             'branch_rating', 'branch_rewiev_number',
             'clinic_page']

work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.head(3)

In [ ]:
work.shape

In [ ]:
work.to_excel(file_path + f'doc_doc_urls_{need_city}.xlsx', encoding='utf8', index=False)

In [ ]:
driver.quit()